In [0]:
print("Hello")

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ItemsDataSQL").getOrCreate()


data = [
    ("FDA15", 9.3, "Low Fat", 0.01604730),
    ("DRC01", 5.92, "Regular", 0.01927821),
    ("FDN15", 17.5, "Low Fat", 0.01676007),
    ("FDX07", 19.2, "Regular", 0.0),
    ("NCD19", 8.93, "Low Fat", 0.0),
    ("FDP36", 10.395, "Regular", 0.0),
    ("FDO10", 13.65, "Regular", 0.01274108),
    ("FDP10", None, "Low Fat", 0.12746985),
    ("FDH17", 16.2, "Regular", 0.01668711),
    ("FDU28", 19.2, "Regular", 0.09444959),
    ("FDY07", 11.8, "Low Fat", 0.0),
    ("FDA03", 18.5, "Regular", 0.04546377),
    ("FDX32", 15.1, "Regular", 0.10001315),
    ("FDS46", 17.6, "Regular", 0.04725732)
]

columns = ["Item_identifier", "Item_Weight", "Item_Fat_Content", "Item_Visibility"]

df_items = spark.createDataFrame(data, columns)

df_items.createOrReplaceTempView("items")

# 1. Show all items
spark.sql("SELECT * FROM items").show()

# 2. Items with weight missing (null)
spark.sql("SELECT * FROM items WHERE Item_Weight IS NULL").show()

# 3. Average item weight by fat content
spark.sql("""
    SELECT Item_Fat_Content, AVG(Item_Weight) AS avg_weight
    FROM items
    GROUP BY Item_Fat_Content
""").show()

# 4. Items sorted by visibility descending
spark.sql("SELECT Item_identifier, Item_Visibility FROM items ORDER BY Item_Visibility DESC").show()

# 5. Count of items by fat content
spark.sql("""
    SELECT Item_Fat_Content, COUNT(*) AS count_items
    FROM items
    GROUP BY Item_Fat_Content
""").show()


In [0]:
catalog = 'main'
schema = 'default'
volume_name = 'vol1'
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog}.{schema}.{volume_name}")

In [0]:
df = spark.read.format('csv').option('inferSchema', True).option('header', True).load("/Volumes/main/default/vol1/BigMart Sales.csv")

display(df.head(5))

##Select

In [0]:
df_select=df.select('Item_Identifier','Item_Weight','Item_Fat_Content').display() 

In [0]:
from pyspark.sql.functions import col
df.select(col('Item_Identifier'),col('Item_Fat_Content'),col('Item_Weight')).display()

## ALIAS


In [0]:
from pyspark.sql.functions import col
df.select(col('Item_Identifier').alias('Item_ID')).display()


## FILTER / WHERE

#### Scenario 1

In [0]:
df.filter(col('Item_Fat_Content')=="Regular").display()

#### Scenario 2

In [0]:
df.filter((col('Item_Type')=='Soft Drinks') & (col('Item_Weight')<10)).display()

#### Scenario 3

In [0]:
df.filter((col('Outlet_Location_Type').isin('Tier 1','Tier 2')) & (col('Outlet_Size').isNull())).display()

## Renaming Column

In [0]:
df.withColumnRenamed('Item_Weight','Weight').display()

## withColumn

#### Scenario 1

In [0]:
from pyspark.sql.functions import lit
df = df.withColumn('flag',lit("new"))
df.display()

In [0]:
df=df.withColumn('multiply',col('Item_Weight')*col('Item_MRP'))
df.display()

#### Scenario 2

In [0]:
from pyspark.sql.functions import regexp_replace
df.withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Regular","Reg")).withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Low Fat","LF")).display()

## Type Casting

In [0]:
from pyspark.sql.types import StringType
df = df.withColumn('Item_Weight',col('Item_Weight').cast(StringType()))
df.display()

## Sort / Order By

#### Scenario 1

In [0]:
df.sort(col('Item_Weight').desc()).display()

#### Scenario 2

In [0]:
df.sort(col('Item_Visibility').asc()).display()

#### Scenario 3

In [0]:
df.sort(['Item_Weight','Item_Visibility'],ascending=[0,0]).display()

In [0]:
df.sort(['Item_Weight','Item_Visibility'],ascending=[0,1]).display()

## Limit

In [0]:
df.limit(10).display()

## DROP


#### SCENARIO 1

In [0]:
df.drop('Item_Visibility').display()

#### SCENARIO 2

In [0]:
df.drop('Item_Visibility','Item_Type').display()

## DROP DUPLICATES

In [0]:
df.dropDuplicates().display()

#### Scenario 2

In [0]:
df.drop_duplicates(subset=['Item_Type']).display()

In [0]:
df.distinct().display()

## Union and Union By Name

#### Preparing Dataframe

In [0]:
data1=[('1','kad'),('2','sid')]
schema1='id STRING, name STRING'
df1=spark.createDataFrame(data1,schema1)
data2 =[('3','rahul'),('4','jas')]
schema2='id STRING, name STRING'
df2=spark.createDataFrame(data2,schema2)


## UNION

In [0]:
df1.union(df2).display()

## UNION BY NAME

In [0]:
df1.unionByName(df2).display()

## STRING FUNCTIONS

#### INITCAP()

In [0]:
from pyspark.sql.functions import initcap
df.select(initcap('Item_Type')).display()

#### LOWER

In [0]:
from pyspark.sql.functions import lower
df.select(lower('Item_Type')).display()

#### UPPER

In [0]:
from pyspark.sql.functions import upper
df.select(upper('Item_Type').alias('Upper_Items')).display()

## DATE Functions

#### Current_Date()
#### Date_Add()

In [0]:
from pyspark.sql.functions import current_date,date_add
df= df.withColumn('current_date',current_date())
df=df.withColumn('add_date',date_add('current_date',3))
df.display()



#### Date_Diff

In [0]:
from pyspark.sql.functions import date_diff
df = df.withColumn('diff_date',date_diff('add_date','current_date'))
df.display()

#### DATE_FORMAT

In [0]:
from pyspark.sql.functions import date_format
df= df.withColumn('current_date',date_format('current_date','dd-MM-yyyy'))
df.display()

## Handling Nulls

#### Dropping Nulls

In [0]:
df.dropna('any').display()

In [0]:
df.dropna(subset=['Outlet_Size']).display()

#### Filling Null Values

#### Scenario 1
#### FillNA

In [0]:
df.fillna('Not Avaialble').display()

In [0]:
df.fillna(0,subset=['Item_Weight']).display()